In [ ]:
#| default_exp 67-1-ngame-ep-for-wikiseealso-with-input-concatenation

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
from scipy import sparse
from tqdm.auto import tqdm
import os,torch, torch.multiprocessing as mp, pickle, numpy as np
from xcai.basics import *
from xcai.models.PPP0XX import DBT009,DBT011

comet_ml is installed but `COMET_API_KEY` is not set.


In [ ]:
os.environ['WANDB_MODE'] = 'disabled'

In [ ]:
#| export
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
os.environ['WANDB_PROJECT']='xc-nlg_66-radga-dr-ep-for-wikiseealso'

### Load data

In [ ]:
#| export
data_dir = '/home/scai/phd/aiz218323/Projects/XC_NLG/data'

In [ ]:
#| export
block = XCBlock.from_cfg(data_dir, 'data_metas', valid_pct=0.001, tfm='xcnlg', tokenizer='distilbert-base-uncased', 
                         smp_features=[('lbl2data',1,1)])

/home/scai/phd/aiz218323/.local/lib/python3.9/site-packages/xclib-0.97-py3.9-linux-x86_64.egg/xclib/data/data_utils.py:263: UserWarning: Header mis-match from inferred shape!
  warnings.warn("Header mis-match from inferred shape!")


In [ ]:
pkl_dir = '/home/scai/phd/aiz218323/scratch/datasets'
pkl_file = f'{pkl_dir}/processed/wikiseealso_data_distilbert-base-uncased_xcnlg_ngame.pkl'
pkl_file = f'{pkl_dir}/processed/wikiseealso_data-metas_distilbert-base-uncased_rm_radga-final.pkl'
pkl_file = f'{pkl_dir}/processed/wikiseealso_data-metas_distilbert-base-uncased_rm_radga-final-aug-hlk.pkl'
pkl_file = f'{pkl_dir}/processed/wikiseealso_data-metas_distilbert-base-uncased_rm_radga-final-aug-cat.pkl'

### Augmentation code

In [ ]:
pkl_dir = '/home/scai/phd/aiz218323/scratch/datasets'

In [ ]:
pkl_file = f'{pkl_dir}/processed/wikiseealso_data-metas_distilbert-base-uncased_rm_radga-final.pkl'

with open(pkl_file, 'rb') as file: block = pickle.load(file)

In [ ]:
block = AugmentMetaInputIdsTfm.apply(block, 'cat_meta', 'data', 32, True)
block = AugmentMetaInputIdsTfm.apply(block, 'cat_meta', 'lbl', 32, True)

In [ ]:
block = AugmentMetaInputIdsTfm.apply(block, 'hlk_meta', 'data', 32, True)
block = AugmentMetaInputIdsTfm.apply(block, 'hlk_meta', 'lbl', 32, True)

In [ ]:
pkl_file = f'{pkl_dir}/processed/wikiseealso_data-metas_distilbert-base-uncased_rm_radga-aug-cat-block-032.pkl'
with open(pkl_file, 'wb') as file: pickle.dump(block, file)

In [ ]:
pkl_file = f'{pkl_dir}/processed/wikiseealso_data-metas_distilbert-base-uncased_rm_radga-aug-cat-block-032.pkl'
with open(pkl_file, 'rb') as file: block = pickle.load(file)

In [ ]:
block.train.dset.data.data_info['input_ids'] = block.train.dset.data.data_info['input_ids_aug_cat']
block.train.dset.data.data_info['attention_mask'] = block.train.dset.data.data_info['attention_mask_aug_cat']
block.test.dset.data.data_info['input_ids'] = block.test.dset.data.data_info['input_ids_aug_cat']
block.test.dset.data.data_info['attention_mask'] = block.test.dset.data.data_info['attention_mask_aug_cat']

In [ ]:
block.train.dset.data.lbl_info['input_ids'] = block.train.dset.data.lbl_info['input_ids_aug_cat']
block.train.dset.data.lbl_info['attention_mask'] = block.train.dset.data.lbl_info['attention_mask_aug_cat']
block.test.dset.data.lbl_info['input_ids'] = block.test.dset.data.lbl_info['input_ids_aug_cat']
block.test.dset.data.lbl_info['attention_mask'] = block.test.dset.data.lbl_info['attention_mask_aug_cat']

In [ ]:
pkl_file = f'{pkl_dir}/processed/wikiseealso_data-metas_distilbert-base-uncased_rm_radga-final-aug-hlk.pkl'
with open(pkl_file, 'wb') as file: pickle.dump(block, file)

In [ ]:
block = AugmentMetaInputIdsTfm.apply(block, 'hlk_meta', 32, True)

block.train.dset.data.data_info['input_ids'] = block.train.dset.data.data_info['input_ids_aug_hlk']
block.train.dset.data.data_info['attention_mask'] = block.train.dset.data.data_info['attention_mask_aug_hlk']

block.test.dset.data.data_info['input_ids'] = block.test.dset.data.data_info['input_ids_aug_hlk']
block.test.dset.data.data_info['attention_mask'] = block.test.dset.data.data_info['attention_mask_aug_hlk']

In [ ]:
pkl_file = f'{pkl_dir}/processed/wikiseealso_data-metas_distilbert-base-uncased_rm_radga-final-aug-hlk.pkl'
with open(pkl_file, 'wb') as file: pickle.dump(block, file)

In [ ]:
pkl_file = f'{pkl_dir}/processed/wikiseealso_data-metas_distilbert-base-uncased_rm_radga-aug-cat-hlk-block-032.pkl'
with open(pkl_file, 'wb') as file: pickle.dump(block, file)

### Augmentation data

In [ ]:
#| export
pkl_dir = '/home/scai/phd/aiz218323/scratch/datasets'
pkl_file = f'{pkl_dir}/processed/wikiseealso_data-metas_distilbert-base-uncased_rm_radga-aug-cat-hlk-block-032.pkl'
with open(pkl_file, 'rb') as file: block = pickle.load(file)

In [ ]:
block.train.dset.data.data_info['input_ids_org'] = block.train.dset.data.data_info['input_ids']
block.train.dset.data.data_info['attention_mask_org'] = block.train.dset.data.data_info['attention_mask']
block.test.dset.data.data_info['input_ids_org'] = block.test.dset.data.data_info['input_ids']
block.test.dset.data.data_info['attention_mask_org'] = block.test.dset.data.data_info['attention_mask']

block.train.dset.data.lbl_info['input_ids_org'] = block.train.dset.data.lbl_info['input_ids']
block.train.dset.data.lbl_info['attention_mask_org'] = block.train.dset.data.lbl_info['attention_mask']
block.test.dset.data.lbl_info['input_ids_org'] = block.test.dset.data.lbl_info['input_ids']
block.test.dset.data.lbl_info['attention_mask_org'] = block.test.dset.data.lbl_info['attention_mask']

In [ ]:
block.train.dset.data.data_info['input_ids'] = block.train.dset.data.data_info['input_ids_org']
block.train.dset.data.data_info['attention_mask'] = block.train.dset.data.data_info['attention_mask_org']
block.test.dset.data.data_info['input_ids'] = block.test.dset.data.data_info['input_ids_org']
block.test.dset.data.data_info['attention_mask'] = block.test.dset.data.data_info['attention_mask_org']

block.train.dset.data.lbl_info['input_ids'] = block.train.dset.data.lbl_info['input_ids_org']
block.train.dset.data.lbl_info['attention_mask'] = block.train.dset.data.lbl_info['attention_mask_org']
block.test.dset.data.lbl_info['input_ids'] = block.test.dset.data.lbl_info['input_ids_org']
block.test.dset.data.lbl_info['attention_mask'] = block.test.dset.data.lbl_info['attention_mask_org']

In [ ]:
#| export
block.train.dset.data.data_info['input_ids'] = block.train.dset.data.data_info['input_ids_aug_cat']
block.train.dset.data.data_info['attention_mask'] = block.train.dset.data.data_info['attention_mask_aug_cat']
block.test.dset.data.data_info['input_ids'] = block.test.dset.data.data_info['input_ids_aug_cat']
block.test.dset.data.data_info['attention_mask'] = block.test.dset.data.data_info['attention_mask_aug_cat']

In [ ]:
#| export
block.train.dset.data.lbl_info['input_ids'] = block.train.dset.data.lbl_info['input_ids_aug_cat']
block.train.dset.data.lbl_info['attention_mask'] = block.train.dset.data.lbl_info['attention_mask_aug_cat']
block.test.dset.data.lbl_info['input_ids'] = block.test.dset.data.lbl_info['input_ids_aug_cat']
block.test.dset.data.lbl_info['attention_mask'] = block.test.dset.data.lbl_info['attention_mask_aug_cat']

In [ ]:
#| export
block.train.dset.data.data_info['input_ids'] = block.train.dset.data.data_info['input_ids_aug_hlk']
block.train.dset.data.data_info['attention_mask'] = block.train.dset.data.data_info['attention_mask_aug_hlk']
block.test.dset.data.data_info['input_ids'] = block.test.dset.data.data_info['input_ids_aug_hlk']
block.test.dset.data.data_info['attention_mask'] = block.test.dset.data.data_info['attention_mask_aug_hlk']

In [ ]:
#| export
block.train.dset.data.lbl_info['input_ids'] = block.train.dset.data.lbl_info['input_ids_aug_hlk']
block.train.dset.data.lbl_info['attention_mask'] = block.train.dset.data.lbl_info['attention_mask_aug_hlk']
block.test.dset.data.lbl_info['input_ids'] = block.test.dset.data.lbl_info['input_ids_aug_hlk']
block.test.dset.data.lbl_info['attention_mask'] = block.test.dset.data.lbl_info['attention_mask_aug_hlk']

In [ ]:
np.mean([len(o) for o in block.test.dset.data.lbl_info['attention_mask']])

6.630477379694554

In [ ]:
np.mean([len(o) for o in block.test.dset.data.data_info['input_ids']])

6.801932231079063

## Training

In [ ]:
args = XCLearningArguments(
    output_dir='/home/scai/phd/aiz218323/scratch/outputs/67-ngame-ep-for-wikiseealso-with-input-concatenation-1-3',
    logging_first_step=True,
    per_device_train_batch_size=800,
    per_device_eval_batch_size=800,
    representation_num_beams=200,
    representation_accumulation_steps=10,
    save_strategy="steps",
    evaluation_strategy="steps",
    eval_steps=3000,
    save_steps=3000,
    save_total_limit=5,
    num_train_epochs=300,
    predict_with_representation=True,
    representation_search_type='INDEX',
    adam_epsilon=1e-6,
    warmup_steps=100,
    weight_decay=0.01,
    learning_rate=2e-4,
    group_by_cluster=True,
    num_clustering_warmup_epochs=10,
    num_cluster_update_epochs=5,
    num_cluster_size_update_epochs=25,
    clustering_type='EXPO',
    minimum_cluster_size=2,
    maximum_cluster_size=1600,
    target_indices_key='plbl2data_idx',
    target_pointer_key='plbl2data_data2ptr',
    use_encoder_parallel=True,
    max_grad_norm=None,
    fp16=True,
)

In [ ]:
metric = PrecRecl(block.n_lbl, block.test.data_lbl_filterer, prop=block.train.dset.data.data_lbl,
                  pk=10, rk=200, rep_pk=[1, 3, 5, 10], rep_rk=[10, 100, 200])

In [ ]:
bsz = max(args.per_device_train_batch_size, args.per_device_eval_batch_size)*torch.cuda.device_count()

model = DBT009.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', bsz=bsz, tn_targ=5000, margin=0.3, tau=0.1, 
                               n_negatives=10, apply_softmax=True, use_encoder_parallel=True)
model.init_dr_head()

Some weights of DBT010 were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['encoder.dr_layer_norm.bias', 'encoder.dr_layer_norm.weight', 'encoder.dr_projector.bias', 'encoder.dr_projector.weight', 'encoder.dr_transform.bias', 'encoder.dr_transform.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
learn = XCLearner(
    model=model, 
    args=args,
    train_dataset=block.train.dset,
    eval_dataset=block.test.dset,
    data_collator=block.collator,
    compute_metrics=metric,
)

/scratch/scai/phd/aiz218323/anaconda3/envs/xc_nlg/lib/python3.9/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
learn.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


  0%|          | 0/347 [00:00<?, ?it/s]

In [ ]:
if __name__ == '__main__':
    mp.freeze_support()
    learn.train()

## Prediction

In [ ]:
#| export
args = XCLearningArguments(
    output_dir='/home/scai/phd/aiz218323/scratch/outputs/67-ngame-ep-for-wikiseealso-with-input-concatenation-1-2',
    logging_first_step=True,
    per_device_train_batch_size=800,
    per_device_eval_batch_size=800,
    representation_num_beams=200,
    representation_accumulation_steps=100,
    predict_with_representation=True,
    representation_search_type='INDEX',
    target_indices_key='plbl2data_idx',
    target_pointer_key='plbl2data_data2ptr',
    use_encoder_parallel=True,
    fp16=True,
)

In [ ]:
#| export
metric = PrecRecl(block.n_lbl, block.test.data_lbl_filterer, prop=block.train.dset.data.data_lbl,
                  pk=10, rk=200, rep_pk=[1, 3, 5, 10], rep_rk=[10, 100, 200])

In [ ]:
#| export
output_dir = f"/home/scai/phd/aiz218323/scratch/outputs/{os.path.basename(args.output_dir)}"
mname = f'{output_dir}/{os.path.basename(get_best_model(output_dir))}'

In [ ]:
#| export
bsz = max(args.per_device_train_batch_size, args.per_device_eval_batch_size)*torch.cuda.device_count()

model = DBT009.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', bsz=bsz, tn_targ=5000, margin=0.3, tau=0.1,
                               n_negatives=10, apply_softmax=True, use_encoder_parallel=True)

Some weights of DBT009 were not initialized from the model checkpoint at sentence-transformers/msmarco-distilbert-base-v4 and are newly initialized: ['encoder.dr_layer_norm.bias', 'encoder.dr_layer_norm.weight', 'encoder.dr_projector.bias', 'encoder.dr_projector.weight', 'encoder.dr_transform.bias', 'encoder.dr_transform.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#| export
from safetensors import safe_open

model_weight_file = f'{mname}/model.safetensors'

model_weights = {}
with safe_open(model_weight_file, framework="pt") as file:
    for k in file.keys(): model_weights[k] = file.get_tensor(k)
        

In [ ]:
#| export
model.load_state_dict(model_weights, strict=False)

_IncompatibleKeys(missing_keys=['distilbert.embeddings.word_embeddings.weight', 'distilbert.embeddings.position_embeddings.weight', 'distilbert.embeddings.LayerNorm.weight', 'distilbert.embeddings.LayerNorm.bias', 'distilbert.transformer.layer.0.attention.q_lin.weight', 'distilbert.transformer.layer.0.attention.q_lin.bias', 'distilbert.transformer.layer.0.attention.k_lin.weight', 'distilbert.transformer.layer.0.attention.k_lin.bias', 'distilbert.transformer.layer.0.attention.v_lin.weight', 'distilbert.transformer.layer.0.attention.v_lin.bias', 'distilbert.transformer.layer.0.attention.out_lin.weight', 'distilbert.transformer.layer.0.attention.out_lin.bias', 'distilbert.transformer.layer.0.sa_layer_norm.weight', 'distilbert.transformer.layer.0.sa_layer_norm.bias', 'distilbert.transformer.layer.0.ffn.lin1.weight', 'distilbert.transformer.layer.0.ffn.lin1.bias', 'distilbert.transformer.layer.0.ffn.lin2.weight', 'distilbert.transformer.layer.0.ffn.lin2.bias', 'distilbert.transformer.layer.

In [ ]:
mname

'/home/scai/phd/aiz218323/scratch/outputs/67-ngame-ep-for-wikiseealso-with-input-concatenation-1-2/checkpoint-130000'

In [ ]:
#| export
learn = XCLearner(
    model=model, 
    args=args,
    train_dataset=block.train.dset,
    eval_dataset=block.test.dset,
    data_collator=block.collator,
    compute_metrics=metric,
)

/scratch/scai/phd/aiz218323/anaconda3/envs/xc_nlg/lib/python3.9/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
#| export
o = learn.predict(block.test.dset)

  0%|          | 0/196 [00:00<?, ?it/s]

/scratch/scai/phd/aiz218323/anaconda3/envs/xc_nlg/lib/python3.9/site-packages/scipy/sparse/_index.py:145: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
display_metric(o.metrics)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,24.6019,16.4339,12.5025,8.0314,24.6019,24.8822,25.8773,27.591,20.471,21.9057,23.6301,27.2401,20.471,22.4529,23.8763,25.7368,32.1282,46.1119,49.3935,0.0296,742.392,239.112,0.15


In [ ]:
#| export
pred_dir = f"{mname}/predictions/"
os.makedirs(pred_dir, exist_ok=True)

with open(f'{pred_dir}/test_predictions.pkl', 'wb') as file: pickle.dump(o, file)


In [ ]:
#| export
print(o.metrics)

In [ ]:
m = {'test_loss': 0.02027089148759842, 'test_P@1': 0.38918401261865193, 'test_P@10': 0.1197121370025047, 'test_P@3': 0.2529326160231173, 'test_P@5': 0.19003239162907803, 'test_N@1': 0.38918399810791016, 'test_N@10': 0.4228399097919464, 'test_N@3': 0.38625404238700867, 'test_N@5': 0.39960503578186035, 'test_PSP@1': 0.2808400762141096, 'test_PSP@10': 0.3843759417211133, 'test_PSP@3': 0.3052513348114733, 'test_PSP@5': 0.331710015691671, 'test_PSN@1': 0.28084006905555725, 'test_PSN@10': 0.3633485436439514, 'test_PSN@3': 0.31347599625587463, 'test_PSN@5': 0.3355429172515869, 'test_R@200': 0.6904824364367135, 'test_R@10': 0.48273773909035944, 'test_R@100': 0.6535670102516896, 'test_runtime': 1090.5389, 'test_samples_per_second': 162.777, 'test_steps_per_second': 0.102}

In [ ]:
display_metric(m)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,38.9184,25.2933,19.0032,11.9712,38.9184,38.6254,39.9605,42.284,28.084,30.5251,33.171,38.4376,28.084,31.3476,33.5543,36.3349,48.2738,65.3567,69.0482,0.0203,1090.5389,162.777,0.102


### `Label-side` Augmentation Analysis

In [ ]:
import re
from xcai.analysis import *
from torch.utils.data import Dataset
from typing import List
from scipy import sparse
import xclib.utils.sparse as xc_sparse

from IPython.display import HTML

In [ ]:
from transformers import AutoTokenizer

In [ ]:
def get_pred(out, n_lbl):
    pred_ptr = torch.concat([torch.zeros((1,), dtype=torch.long), out.pred_ptr.cumsum(dim=0)])
    return sparse.csr_matrix((out.pred_score, out.pred_idx, pred_ptr), shape=(len(out.pred_ptr), n_lbl))


In [ ]:
tokz = AutoTokenizer.from_pretrained('distilbert-base-uncased')

pred = get_pred(o, block.n_lbl)
top_pred = xc_sparse.retain_topk(pred, k=10)

pred_dset = get_pred_dset(top_pred, block)

In [ ]:
aug_cat_input_text = tokz.batch_decode(pred_dset.data.lbl_info['input_ids_aug_cat'], skip_special_tokens=True)

In [ ]:
pred_dset.data.lbl_info['aug_cat_input_text'] = aug_cat_input_text
block.test.dset.data.data_info['aug_cat_input_text'] = aug_cat_input_text

In [ ]:
class TextColumns(Dataset):
    
    def __init__(self, x, pat='.*_text$'):
        self.x, self.pat = x, pat
    
    def __getitem__(self, idx):
        o = self.x[idx]
        return {k:v for k,v in o.items() if re.match(self.pat, k)}
    

In [ ]:
pattern = r'^(data|lbl2data|lbl2data_aug_cat)_input_text$'

pred_dset = TextColumns(pred_dset, pat=pattern)
test_dset = TextColumns(block.test.dset, pat=pattern)

In [ ]:
evals = pointwise_eval(pred, block.test.dset.data.data_lbl, topk=5, metric='P', return_type='D')

In [ ]:
idxs = np.argsort(evals)[-10:]

In [ ]:
def display_text(pred_dset:Dataset, data_dset:Dataset, idxs:List):
    color = [('red','green'), ('black','blue')]
    text = []
    for i,idx in enumerate(idxs):
        c = color[i%len(color)]
        pred_text = "<br>".join([f'{html(k,color[0][0])}: {html(v,color[0][1])}' for k,v in pred_dset[idx].items()])
        data_text = "<br>".join([f'{html(k,color[1][0])}: {html(v,color[1][1])}' for k,v in test_dset[idx].items()])
        text.append("<br>".join([pred_text,data_text]))
    return "<br><br>".join(text)

In [ ]:
HTML(display_text(pred_dset, test_dset, idxs))

In [ ]:
HTML(display_text(pred_dset, test_dset, idxs))

## `Zero-shot`

In [ ]:
from xcai.data import MainXCData,MainXCDataset,MetaXCData,MetaXCDataset,XCDataset,BaseXCDataBlock,XCDataBlock

In [ ]:
data_dir = '/home/scai/phd/aiz218323/Projects/XC_NLG/data/(mapped)LF-WikiSeeAlsoTitles-320K'
data_cfg = {
    'info_column_names': ['identifier', 'input_text'],
    'use_tokenizer': True,
    'tokenizer': 'bert-base-uncased',
    'tokenization_column': 'input_text',
    'max_sequence_length': 32,
}

In [ ]:
test_cfg = {
    'data_lbl': f'{data_dir}/tst.0_X_Y.txt',
    'data_info': f'{data_dir}/raw_data/test.0.raw.txt',
    'lbl_info': f'{data_dir}/raw_data/label.0.raw.txt',
}

In [ ]:
test_data = MainXCData.from_file(**test_cfg, **data_cfg)
test_main = MainXCDataset(**test_data)

### Metadata re-mapping

In [ ]:
train_id, train_idx, zero_train_idx = np.intersect1d(block.train.dset.data.data_info['identifier'], 
                                                     test_dset.data_info['identifier'], return_indices=True)

In [ ]:
train_meta = block.train.dset.meta['hlk_meta'].data_meta[train_idx]

In [ ]:
test_id, test_idx, zero_test_idx = np.intersect1d(block.test.dset.data.data_info['identifier'], 
                                                  test_dset.data_info['identifier'], return_indices=True)

In [ ]:
test_meta = block.test.dset.meta['hlk_meta'].data_meta[test_idx]

In [ ]:
zero_meta = sparse.csr_matrix((test_dset.n_data, block.train.dset.meta['hlk_meta'].n_meta))

In [ ]:
for i in tqdm(range(len(zero_train_idx))): zero_meta[zero_train_idx[i]] = train_meta[i]

  0%|          | 0/39079 [00:00<?, ?it/s]

/scratch/scai/phd/aiz218323/anaconda3/envs/xc_nlg/lib/python3.9/site-packages/scipy/sparse/_index.py:136: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray_sparse(i, j, x)


In [ ]:
for i in tqdm(range(len(zero_test_idx))): zero_meta[zero_test_idx[i]] = test_meta[i]

  0%|          | 0/22982 [00:00<?, ?it/s]

In [ ]:
zero_meta

<127082x2458399 sparse matrix of type '<class 'numpy.float64'>'
	with 2170055 stored elements in Compressed Sparse Row format>

In [ ]:
import xclib.data.data_utils as du

In [ ]:
fname = f'{data_dir}/hyper_link_tst_0_X_Y.txt'
du.write_sparse_file(zero_meta, fname)

### Load metadata

In [ ]:
meta_cfg = {
    'prefix': 'cat',
    'data_meta': f'{data_dir}/category_tst_0_X_Y.txt',
    'lbl_meta': f'{data_dir}/category_lbl_0_X_Y.txt',
    'meta_info': f'{data_dir}/raw_data/category.raw.txt',
}
meta_data = MetaXCData.from_file(**meta_cfg, **data_cfg)

/home/scai/phd/aiz218323/.local/lib/python3.9/site-packages/xclib-0.97-py3.9-linux-x86_64.egg/xclib/data/data_utils.py:263: UserWarning: Header mis-match from inferred shape!
  warnings.warn("Header mis-match from inferred shape!")


In [ ]:
meta_main = MetaXCDataset(**meta_data)

In [ ]:
test_dset = XCDataset(test_main, cat_meta=meta_main)

In [ ]:
idx = np.where(test_block.test.dset.meta.cat_meta.data_meta.getnnz(axis=1) > 0)[0]
sampled_test_dset = test_dset._getitems(idx)

In [ ]:
test_base_block = BaseXCDataBlock(sampled_test_dset, collate_fn=block.collator)
test_block = XCDataBlock(test=test_base_block)

In [ ]:
test_block = AugmentMetaInputIdsTfm.apply(test_block, 'cat_meta', 'data', 32, True)

In [ ]:
test_block.test.dset.data.data_info['input_ids'] = test_block.test.dset.data.data_info['input_ids_aug_cat']
test_block.test.dset.data.data_info['attention_mask'] = test_block.test.dset.data.data_info['attention_mask_aug_cat']

In [ ]:
test_block.test.dset.meta.cat_meta.data_meta.getnnz(axis=1).mean()

5.950017798776739

In [ ]:
np.mean([len(o) for o in test_block.test.dset.data.data_info['attention_mask']])

27.245461311931653

In [ ]:
from xcai.analysis import *
TextColumns(test_dset)[2000]

{'data_input_text': 'Paraguay',
 'lbl2data_input_text': ['Outline of Paraguay'],
 'cat2data_input_text': ['Landlocked countries',
  'Member states of the United Nations',
  'Republics',
  'Former Spanish colonies',
  'Spanish-speaking countries and territories',
  'Countries in South America',
  'Member states of Mercosur',
  'States and territories established in 1811',
  'Paraguay',
  '1811 establishments in South America'],
 'cat2lbl2data_input_text': [[]]}

### Inference

In [ ]:
mdir = ['59-ngame-ep-for-wikiseealso-with-cls-for-dr-1-1', '67-ngame-ep-for-wikiseealso-with-input-concatenation-1-1',
        '67-ngame-ep-for-wikiseealso-with-input-concatenation-1-2', '67-ngame-ep-for-wikiseealso-with-input-concatenation-1-3']

In [ ]:
args = XCLearningArguments(
    output_dir=f'/home/scai/phd/aiz218323/scratch/outputs/{mdir[1]}',
    logging_first_step=True,
    per_device_train_batch_size=800,
    per_device_eval_batch_size=800,
    representation_num_beams=200,
    representation_accumulation_steps=100,
    predict_with_representation=True,
    representation_search_type='BRUTEFORCE',
    target_indices_key='plbl2data_idx',
    target_pointer_key='plbl2data_data2ptr',
    use_encoder_parallel=True,
    fp16=True,
)

In [ ]:
metric = PrecRecl(test_block.test.dset.n_lbl, test_block.test.dset.data.data_lbl_filterer, prop=block.train.dset.data.data_lbl,
                  pk=10, rk=200, rep_pk=[1, 3, 5, 10], rep_rk=[10, 100, 200])

In [ ]:
output_dir = f"/home/scai/phd/aiz218323/scratch/outputs/{os.path.basename(args.output_dir)}"
mname = f'{output_dir}/{os.path.basename(get_best_model(output_dir))}'

In [ ]:
bsz = max(args.per_device_train_batch_size, args.per_device_eval_batch_size)*torch.cuda.device_count()

model = DBT009.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', bsz=bsz, tn_targ=5000, margin=0.3, tau=0.1,
                               n_negatives=10, apply_softmax=True, use_encoder_parallel=True)

Some weights of DBT009 were not initialized from the model checkpoint at sentence-transformers/msmarco-distilbert-base-v4 and are newly initialized: ['encoder.dr_layer_norm.bias', 'encoder.dr_layer_norm.weight', 'encoder.dr_projector.bias', 'encoder.dr_projector.weight', 'encoder.dr_transform.bias', 'encoder.dr_transform.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from safetensors import safe_open

model_weight_file = f'{mname}/model.safetensors'

model_weights = {}
with safe_open(model_weight_file, framework="pt") as file:
    for k in file.keys(): model_weights[k] = file.get_tensor(k)
        

In [ ]:
model.load_state_dict(model_weights, strict=False)

_IncompatibleKeys(missing_keys=['distilbert.embeddings.word_embeddings.weight', 'distilbert.embeddings.position_embeddings.weight', 'distilbert.embeddings.LayerNorm.weight', 'distilbert.embeddings.LayerNorm.bias', 'distilbert.transformer.layer.0.attention.q_lin.weight', 'distilbert.transformer.layer.0.attention.q_lin.bias', 'distilbert.transformer.layer.0.attention.k_lin.weight', 'distilbert.transformer.layer.0.attention.k_lin.bias', 'distilbert.transformer.layer.0.attention.v_lin.weight', 'distilbert.transformer.layer.0.attention.v_lin.bias', 'distilbert.transformer.layer.0.attention.out_lin.weight', 'distilbert.transformer.layer.0.attention.out_lin.bias', 'distilbert.transformer.layer.0.sa_layer_norm.weight', 'distilbert.transformer.layer.0.sa_layer_norm.bias', 'distilbert.transformer.layer.0.ffn.lin1.weight', 'distilbert.transformer.layer.0.ffn.lin1.bias', 'distilbert.transformer.layer.0.ffn.lin2.weight', 'distilbert.transformer.layer.0.ffn.lin2.bias', 'distilbert.transformer.layer.

In [ ]:
learn = XCLearner(
    model=model, 
    args=args,
    eval_dataset=test_block.test.dset,
    data_collator=block.collator,
    compute_metrics=metric,
)

/scratch/scai/phd/aiz218323/anaconda3/envs/xc_nlg/lib/python3.9/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
o = learn.predict(test_block.test.dset)

  0%|          | 0/53 [00:00<?, ?it/s]

In [ ]:
display_metric(o.metrics)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,22.1935,11.0703,7.4033,4.136,22.1935,27.1571,28.5588,29.812,21.9039,29.4037,32.5719,36.3034,21.9039,27.1897,28.5877,29.8319,37.772,46.5246,48.7688,0.0302,43.8177,1410.434,0.89
